<a href="https://colab.research.google.com/github/wanjiku-wairimu1/Deep-Learning/blob/main/ClassSpeechToText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import os
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
from tqdm import tqdm


In [ ]:
# Subsample
labels = [
    'down', 'left', 'up',
]
train_audio_path = '/content/drive/MyDrive/audio'

all_wave = []
all_label = []
for label in tqdm(labels):
    waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]
    for wav in waves:
        samples, sample_rate = librosa.load(train_audio_path + '/' + label + '/' + wav, sr = 16000)
        samples = librosa.resample(samples, sample_rate, 8000)
        if(len(samples)== 8000) : 
            all_wave.append(samples)
            all_label.append(label)

100%|██████████| 3/3 [03:00<00:00, 60.23s/it]


In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

label_enconder = LabelEncoder()
y = label_enconder.fit_transform(all_label)
classes = list(label_enconder.classes_)
classes
y = np_utils.to_categorical(y, num_classes=len(labels))

In [ ]:
all_wave = np.array(all_wave).reshape(-1,8000,1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(np.array(all_wave),np.array(y),stratify=y,test_size = 0.2,random_state=777,shuffle=True)

In [ ]:
from keras.layers import Bidirectional, BatchNormalization, CuDNNGRU, TimeDistributed

In [ ]:
from keras.layers import Dense, Dropout, Flatten, Conv1D, Input, MaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
K.clear_session()

inputs = Input(shape=(8000,1))
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True)(inputs)

#First Conv1D layer
x = Conv1D(8,13, padding='valid', activation='relu', strides=1)(x)
x = MaxPooling1D(3)(x)
x = Dropout(0.3)(x)

#Second Conv1D layer
x = Conv1D(16, 11, padding='valid', activation='relu', strides=1)(x)
x = MaxPooling1D(3)(x)
x = Dropout(0.3)(x)

#Third Conv1D layer
x = Conv1D(32, 9, padding='valid', activation='relu', strides=1)(x)
x = MaxPooling1D(3)(x)
x = Dropout(0.3)(x)

x = BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True)(x)
x = Bidirectional(CuDNNGRU(128, return_sequences=True), merge_mode='sum')(x)
x = Bidirectional(CuDNNGRU(128, return_sequences=True), merge_mode='sum')(x)
x = Bidirectional(CuDNNGRU(128, return_sequences=False), merge_mode='sum')(x)

x = BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True)(x)

#Flatten layer
# x = Flatten()(x)

#Dense Layer 1
x = Dense(256, activation='relu')(x)
outputs = Dense(len(labels), activation="softmax")(x)

model = Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8000, 1)]         0         
                                                                 
 batch_normalization (BatchN  (None, 8000, 1)          4         
 ormalization)                                                   
                                                                 
 conv1d (Conv1D)             (None, 7988, 8)           112       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 2662, 8)          0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 2662, 8)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 2652, 16)          1424  

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='nadam',metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', 
                           verbose=1, patience=10, min_delta=0.0001)

checkpoint = ModelCheckpoint('speech2text_model.hdf5', monitor='val_acc', 
                             verbose=1, save_best_only=True, mode='max')

In [ ]:
hist = model.fit(
    x=x_train, 
    y=y_train,
    epochs=3, 
    callbacks=[early_stop, checkpoint], 
    batch_size=32, 
    validation_data=(x_valid,y_valid)
)

Epoch 1/3
226/226 [==============================] - ETA: 0s - loss: 1.0656 - accuracy: 0.4579

226/226 [==============================] - 36s 89ms/step - loss: 1.0656 - accuracy: 0.4579 - val_loss: 1.3132 - val_accuracy: 0.4184
Epoch 2/3
225/226 [============================>.] - ETA: 0s - loss: 0.4511 - accuracy: 0.8204

226/226 [==============================] - 19s 84ms/step - loss: 0.4513 - accuracy: 0.8204 - val_loss: 0.9407 - val_accuracy: 0.5105
Epoch 3/3
225/226 [============================>.] - ETA: 0s - loss: 0.2231 - accuracy: 0.9161

226/226 [==============================] - 19s 85ms/step - loss: 0.2230 - accuracy: 0.9162 - val_loss: 0.2503 - val_accuracy: 0.9034


In [ ]:
model.save('speech2text_model.hdf5')

In [ ]:
from keras.models import load_model
model = load_model('speech2text_model.hdf5')

In [ ]:
def s2t_predict(audio, shape_num=8000):
    prob = model.predict(audio.reshape(1,-1,1))   #audio.reshape((1, -1, 1))
    index = np.argmax(prob[0])
    return classes[index]

In [ ]:
audio, audio_rate = librosa.load('01bb6a2a_nohash_3.wav')
audio_sample = librosa.resample(audio, audio_rate, 8000)
ipd.Audio(audio_sample,rate=8000)
audio_sample


array([-0.03714728, -0.06518734, -0.07528991, ..., -0.06925637,
       -0.07490504, -0.08470757], dtype=float32)

In [ ]:
print("Predicted Text:", s2t_predict(audio_sample))

Predicted Text: zero
